In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader


In [2]:
df = pd.read_csv('Iris.csv')
df = df.drop("Id", axis=1)
df.head(10)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [3]:
X = df.drop('Species', axis=1).values
y = df['Species'].values

In [4]:
le = LabelEncoder()
y = le.fit_transform(y)  

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [8]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

In [9]:
class IrisMLP(nn.Module):
    def __init__(self):
        super(IrisMLP,self).__init__()
        self.fc1 = nn.Linear(4,8)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(8,3)

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
model = IrisMLP()

In [10]:
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [11]:
num_epochs = 50  # Rename to avoid collision

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [10/50], Loss: 0.1639
Epoch [20/50], Loss: 0.0065
Epoch [30/50], Loss: 0.0072
Epoch [40/50], Loss: 0.0022
Epoch [50/50], Loss: 0.0324


In [12]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_X,batch_y in test_loader:
        outputs = model(batch_X)
        _,predicted = torch.max(outputs.data,1)
        total +=batch_y.size(0)
        correct += (predicted==batch_y).sum().item()
print(f"\n Test Accuracy:{100*correct/total:.2f}%")


 Test Accuracy:100.00%
